In [2]:
import pandas as pd

In [62]:
# Get all sheet names
sheet_names = pd.ExcelFile("Financials data.xlsx").sheet_names

# Initialize empty list to store dataframes
dfs = []

# Process each sheet
for sheet in sheet_names:
    # Read the sheet
    temp_df = pd.read_excel("Financials data.xlsx", sheet_name=sheet)
    
    # Apply the same transformations
    temp_df = temp_df.T.iloc[1:, :110]
    temp_df.columns = temp_df.iloc[0]
    temp_df = temp_df.iloc[1:].reset_index(drop=True)
    
    # Add company column
    temp_df['Company'] = sheet
    
    # Append to list
    dfs.append(temp_df)

# Combine all dataframes
df = pd.concat(dfs, ignore_index=True)

In [63]:
df = df[["Revenue", "- Cost of Revenue", "Reported R&D Expense (from Footnote)", "Cost of Goods Sold",
          "Inventory - Raw Materials", "Inventory - Work in Progress", "Inventory - Finished Goods",
          "Selling and marketing Expense", "Period","Company"]]

In [64]:
df["Cost of Revenue / Revenue"] = df["- Cost of Revenue"] / df["Revenue"]
df["Selling and marketing Expense / Revenue"] = df["Selling and marketing Expense"] / df["Revenue"]
df["Reported R&D Expense (from Footnote) / Revenue"] = df["Reported R&D Expense (from Footnote)"] / df["Revenue"]
df["Cost of Goods Sold / Revenue"] = df["Cost of Goods Sold"] / df["Revenue"]
df["Total Inventory"] = df["Inventory - Raw Materials"] + df["Inventory - Work in Progress"] + df["Inventory - Finished Goods"]
df["Inventory - Raw Materials / Total Inventory"] = df["Inventory - Raw Materials"] / df["Total Inventory"]
df["Inventory - Work in Progress / Total Inventory"] = df["Inventory - Work in Progress"] / df["Total Inventory"]
df["Inventory - Finished Goods / Total Inventory"] = df["Inventory - Finished Goods"] / df["Total Inventory"]

df.fillna(0, inplace=True)

df_client = df[df["Company"]=="ClientCo"]
df_competitor = df[df["Company"]!="ClientCo"]

C:\Users\charl\AppData\Local\Temp\ipykernel_8084\2527782014.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


In [65]:
df_client = df_client.iloc[-1]
df_client.drop(columns=["Company", "Period"], inplace=True)
df_competitor = df_competitor.groupby("Company").tail(1)
df_competitor.drop(columns=["Company", "Period"], inplace=True)
df_competitor = df_competitor.mean()

In [66]:
final_df = pd.DataFrame({"ClientCo": df_client, "Competitor Average": df_competitor})

In [67]:
final_df

,ClientCo,Competitor Average
Unnamed: 1,,
- Cost of Revenue,11881.532,60616.574631
Company,ClientCo,NaN
Cost of Goods Sold,11881.532,60616.574631
Cost of Goods Sold / Revenue,0.75262,0.771881
Cost of Revenue / Revenue,0.75262,0.771881
Inventory - Finished Goods,0.0,1624.250319
Inventory - Finished Goods / Total Inventory,0.0,0.08458
Inventory - Raw Materials,0.0,238.46854
Inventory - Raw Materials / Total Inventory,0.0,0.037489
